# Source Morphology

We will know take a deeper look at the morphology of the emission. For this we will use the map data set that we have created earlier. We will fit a spatial and spectral model simultanously to the data.

## Imports
We will start with the import of a few modules.

In [ ]:
import matplotlib.pyplot as plt

import numpy as np

from numpy import sqrt

import astropy.units as u

from astropy.coordinates import (
    SkyCoord, 
    Angle,
)

from gammapy.stats import WStatCountsStatistic

from gammapy.maps import Map

from gammapy.datasets import MapDatasetOnOff

from gammapy.modeling.models import (
    Models,
    SkyModel,
    PowerLawSpectralModel,
    ExpCutoffPowerLawSpectralModel,
    PointSpatialModel,
    GaussianSpatialModel
)

In [ ]:
from gammapy.estimators import ExcessMapEstimator

In [ ]:
from gammapy.modeling import Fit

fit = Fit()

## Load Dataset

We use the MapDataset that we have created earlier.

In [ ]:
map_stacked = MapDatasetOnOff.read('MapDataset.fits.gz')

In [ ]:
map_stacked.peek()

We will need the source position of our object. It is probably in the centre of the dataset:

In [ ]:
source_pos = map_stacked.geoms['geom'].center_skydir

In [ ]:
source_pos

We do not need the full dataset. We will work only on a part, a cutout of the original dataset:

In [ ]:
map_cutout = map_stacked.cutout(source_pos, width = 1.5*u.deg)

In [ ]:
map_cutout.peek()

We will make use of our best-fit spectral model. Let's load this as well:

In [ ]:
bestfit_spectrum = Models.read('SpectrumBestFit.fits.gz')

In [ ]:
bestfit_spectrum

## Sky Model

We will now create a sky model. This is a model describing the energy distribution (a spectral model, as before) and the spatial distribution of the data.

### Point-Like Source

We will start with point-like source, located at the position of our object of interest.

In [ ]:
spatial_point = PointSpatialModel.from_position(source_pos)

In [ ]:
spatial_point.plot(add_cbar = True)

For the spectral model we will use a copy of our model obtained in the last session.

In [ ]:
spectral_point = bestfit_spectrum[0].spectral_model.copy()

Now we create a model combining the spectral and spatial model:

In [ ]:
model_point = SkyModel(spectral_model = spectral_point,
                       spatial_model = spatial_point,
                       name = 'point-like'
                      )

Now we can fit the model:

In [ ]:
map_cutout.models = model_point

fit.run(map_cutout)

In [ ]:
map_cutout.plot_residuals(kwargs_spatial = 
                          {'cmap' : 'plasma',
                           'vmin' : -0.5, 
                           'vmax' : 0.5})

We can also calculate the distance of the best-fit position from our nominal position:

In [ ]:
spatial_point.position.separation(source_pos).to_string(decimal = True)

We should consider to use this new position in our spectrum fit.

As before we can also make a significance sky map, which will now subtract our bestf-it model. We will be left with the residuals of our fit, so that we can check if the model describes the data well or if there are any other sources in the field of view.

In [ ]:
estimator = ExcessMapEstimator(0.05*u.deg)

fluxmaps = estimator.run(map_cutout)

In [ ]:
fluxmaps.sqrt_ts.plot(add_cbar = True,
                      cmap = 'plasma')

And we can keep the test statistics to compare different models later.

In [ ]:
TS_point = map_cutout.stat_sum()

In [ ]:
TS_point

### Gaussian morphology

In [ ]:
spatial_gauss = GaussianSpatialModel.from_position(source_pos)

In [ ]:
spatial_gauss.parameters.to_table()

The eccentricity ```e``` and angle ```phi``` are frozen for the moment. Freeing these parameters will allow the fit of an asymmetric emission.

In [ ]:
spatial_gauss.plot(add_cbar = True)

In [ ]:
spectral_gauss = bestfit_spectrum[0].spectral_model.copy()

Now we create a model combining the spectral and spatial model:

In [ ]:
model_gauss = SkyModel(spectral_model = spectral_gauss,
                       spatial_model = spatial_gauss,
                       name = 'gaussian'
                      )

In [ ]:
map_cutout.models = model_gauss

fit.run(map_cutout)

In [ ]:
map_cutout.plot_residuals(kwargs_spatial = 
                          {'cmap' : 'plasma',
                           'vmin' : -0.5, 
                           'vmax' : 0.5})

In [ ]:
spatial_gauss.position.separation(source_pos).to_string(decimal = True)

In [ ]:
model_gauss.parameters.to_table()

In [ ]:
fluxmaps = estimator.run(map_cutout)

In [ ]:
fluxmaps.sqrt_ts.plot(add_cbar = True,
                      cmap = 'plasma')

In [ ]:
TS_gauss = map_cutout.stat_sum()

In [ ]:
TS_gauss

We do find a slight extension. We should check if this is significant.

### Compare the Models

In [ ]:
print(TS_point, TS_gauss)

In [ ]:
TS = TS_point-TS_gauss
print(TS)

In [ ]:
import scipy.stats

In [ ]:
P = scipy.stats.chi2.sf(TS,1)

print('probabilty: ',P)

In [ ]:
print('significant?', P < 2.7e-3)

So this is not at all significant. We keep the point-source model. But we should consider to re-do the spectral analysis at the best-fit position.

In [ ]:
bestfit_spatial = model_point

In [ ]:
map_cutout.models = bestfit_spatial

### your playground

You can free the eccentricity and angle to check if the emission is extended and asymmetric.
You can also try different spatial models. You will find a list here:

[https://docs.gammapy.org/1.2/user-guide/model-gallery/index.html#model-gallery](https://docs.gammapy.org/1.2/user-guide/model-gallery/index.html#model-gallery)

## Spectrum Comparison

Here we have fit the spectrum again. The fit should not have changed a lot, but we can compare.

In [ ]:
from gammapy.modeling.models import create_crab_spectral_model

In [ ]:
fig, ax = plt.subplots()

plot_kwargs = {
    "energy_bounds": [0.1, 30] * u.TeV,
    "ax": ax,
}

bestfit_spectrum[0].spectral_model.plot(**plot_kwargs, label="spectral fit")
bestfit_spectrum[0].spectral_model.plot_error(facecolor="blue", alpha=0.3, **plot_kwargs)

bestfit_spatial.spectral_model.plot(**plot_kwargs, label="morphology fit")
bestfit_spatial.spectral_model.plot_error(facecolor="red", alpha=0.3, **plot_kwargs)


create_crab_spectral_model("hess_pl").plot(
    **plot_kwargs,
    label="Crab reference",
)

ax.legend()

## Another look at the significance map

We can load our significance map again and plot the best-fit position and error.

In [ ]:
significance_map = Map.read('SigMap.fits.gz')

In [ ]:
zoomed_significance_map = significance_map.cutout(source_pos, 0.5*u.deg)

In [ ]:
ax = zoomed_significance_map.plot(add_cbar = True,
                                  cmap = 'plasma'
                                  )

bestfit_spatial.spatial_model.to_region().to_pixel(ax.wcs).plot(ax=ax, 
                                                                color = 'black'
                                                               )

bestfit_spatial.spatial_model.plot_position_error(ax)

## Summary

We have modelled the emission with two different hypotheses for the spatial distribution. This study can become very complicated, in particular when having several sources, or when there is no simple model to describe the emission. The result of this study may need to be considered when doing the spectral analysis.